# Problem Set 2 (Total: 34 points)

This problem set will give you practice with:

- Data wrangling with pandas: groupby, subsetting, sorting, etc.
- Defining your own functions
- Visualizing trends in data

We're going to investigate different types of disparities in sentencing between Black defendants and White defendants using the same Cook County sentencing dataset as for problem set 1. This analysis could be extended to study Hispanic defendants or, in a different jurisdiction, Asian and other minoritized groups.

As before, this dataset reports the sentence given to defendants convicted of different crimes, and you can find [the data codebook here](https://datacatalog.cookcountyil.gov/api/views/tg8v-tm6u/files/8597cdda-f7e1-44d1-b0ce-0a4e43f8c980?download=true&filename=CCSAO%20Data%20Glossary.pdf) and the latest on these data [at the official website](https://datacatalog.cookcountyil.gov/Courts/Sentencing/tg8v-tm6u).

**Details if interested in digging deeper** (optional): There is a lot to think about here in terms of (1) how we might measure disparities, and (2) what factors you would want to adjust for when deciding whether two defendants are 'similarly situated' but for their race. You can read more technical coverage in the following sources:

- [Review of sentencing disparities research](https://www.journals.uchicago.edu/doi/full/10.1086/701505)
- [Discussion of causal model/blinding race at charging stage of the prosecutorial process](https://5harad.com/papers/blind-charging.pdf)
- [Discussion of measuring discrimination in policing that can generalize to the sentencing case](https://www.annualreviews.org/doi/abs/10.1146/annurev-criminol-011518-024731)
- [General discussion of causal challenges in measuring between-group disparities](https://osf.io/preprints/socarxiv/gx4y3/)

**One major caveat**: when comparing whether two similar defendants received different sentences, we're missing one important attribute that influences sentencing: the defendant's criminal history. This influences sentencing both through sentencing guidelines, which can prescribe longer sentences for those who have certain types of prior convictions, and through judicial discretion if judges are more lenient with first-time defendants. The above sources discuss how much we want to "control away" for this prior history, since if we think there are racial biases in which defendants, conditional on *committing* a crime, are arrested and charged, we may not want to adjust for that factor.

# 0. Load packages and imports

In [ ]:
## basic functionality
import pandas as pd
import numpy as np
import re

## for plotting; can also use seaborn
## note: for plotnine, you likely need to install using pip or conda
import plotnine
from plotnine import *
import matplotlib.pyplot as plt

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## datetime util
from dateutil.relativedelta import relativedelta

## 0.1 Load the data (0 points)

Use `pd.read_csv` to load the `sentencing_cleaned.csv` data. This file is included in the same zip file we used for pset1; if you haven't already, you'll need to unzip `pset1_inputdata.zip` (try the `unzip` shell command). When loading in the data, make sure you're pointing to the right directory and *don't* hard-code your user-specific path name--instead, use relative paths with `../` to go up a level if/when needed.

You'll notice the data is slightly different from what we used for pset 1. 

**Hint:**
You may receive a warning about mixed data types upon loading the .csv file into pandas; feel free to ignore, or call `low_memory=False` as a parameter.

**Concepts tested and resources:** 

- Filepaths and moving around in terminal, as covered in [the lecture and activities on workflow tools](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/04_qss20_f22_workflow.pdf)

In [ ]:
# your code here

# 1. Investigate Black vs. White sentencing disparities

## 1.1 Investigating one type of between-group difference: who reaches the sentencing stage? (5 points)

Calculate the fraction of Black versus White defendants by month and year:

- Denominator is number of unique cases that month
- Numerator for black defendants is count of `is_black_derived`
- Numerator for white defendants is count of `is_white_derived`
- Fraction of each is numerator/denominator

**Hint:**
For this and other time-based questions in this pset, you can use either `sentenceym_derived` or `sentenceymd_derived` (whichever makes more sense for the question).

**Concepts tested and resources**:
- Groupby and agg, as covered in [these lecture slides on data wrangling](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/02_qss20_f22_pandas.pdf), [the class activity on data wrangling](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/00_pandas_datacleaning_blank.ipynb) and its [solutions](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/00_pandas_datacleaning_solutions.ipynb), and [the DataCamp on Data Manipulation with Pandas](https://app.datacamp.com/learn/courses/data-manipulation-with-pandas)

- List comprehension (one option), as covered in [these lecture slides on lists & functions](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/03_qss20_f22_userdefinedfunctions.pdf)


In [ ]:
# your code here

- With that calculation, create a graph with two lines--- one for Black defendants as fraction of total; another for White defendants. Make sure it includes a legend summarizing which color is for which group, and clean the legend so that it has informative names (e.g., Black or White rather than prop_black or prop_white)
- Use mathematical notation to write out each of the proportions using summation notation in a 1-2 sentence writeup describing trends. What seems to be going on in April and May 2020? 

**Optional challenge** (no extra credit points): improve the viz by shading the background of the visualization for months with fewer than 100 cases 

**Optional challenge** (no extra credit points): improve the viz by adding a vertical line for 12-01-2016, the month that new State's Attorney Foxx took office 

**Hints:**

- Access mathematical notation in Jupyter notebooks with the dollar sign (`$`) special character and commands like `\dfrac{Numerator}{Denominator}` and `\sum_{start}^{end}`, e.g.: 
$\dfrac{\sum_{i}^{N} One thing}{\sum_{i}^{N} Another thing}$

**Concepts tested and resources:** 

- Visualization, as covered in [this plotnine example code](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/02_plottingexamples_plotnine.ipynb), chapter 4 of [the DataCamp course on Data Manipulation with Pandas](https://app.datacamp.com/learn/courses/data-manipulation-with-pandas), and the optional DataCamp courses on Data Visualization with ggplot2/Matplotlib

In [ ]:
# your code here

(Your interpretation here)

## 1.2 Investigating the first type of disparity: probation versus incarceration (9 points)

One type of disparity beyond who arrives at the sentencing stage is whether the defendant receives probation or incaceration.

According to the codebook, incarceration is indicated by `COMMITMENT_TYPE` == "Illinois Department of Corrections"

Recreate the previous plot but where the y axis represents the difference between the following proportions (can be either Black - White or White - Black but make sure to label):

- Percent of black defendants who are incarcerated out of all black defendants that month/year 
- Percent of white defendants who are incarcerated out of all white defendants that month/year 

The resulting line should reflect the disparities in rates between black and white. So, for instance, if the Black incarceration rate is 60% and White is 40%, the line would be at 20% (if looking at Black - white). 

Briefly interpret the results.

**Extra credit** (1 point): Improve the viz by plotting a smoothed/best-fit version of the above line, for a total of **two lines** in a single plot.

**Hints**: 

- There are lots of ways to prepare your data in a way that swaps axes. One option is to flip the table with [`pd.pivot()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot.html), rename the columns, then divide the number incarcerated for each race by total number of cases for that race. 

- To rename columns, you can use this general syntax: `df.columns = ['sentences', 'black count', 'white count']`. Make sure the length of colnames list is equivalent to number of cols in the aggregated data you’re working with.

**Concepts tested and resources:** 

- Groupby with agg and row subsetting with logical conditions (e.g., with np.where), as covered in [these lecture slides on data wrangling](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/02_qss20_f22_pandas.pdf), [the class activity on data wrangling](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/00_pandas_datacleaning_blank.ipynb) and its [solutions](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/00_pandas_datacleaning_solutions.ipynb), and [the DataCamp on Data Manipulation with Pandas](https://app.datacamp.com/learn/courses/data-manipulation-with-pandas)

- Visualization, as covered in [this plotnine example code](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/02_plottingexamples_plotnine.ipynb), chapter 4 of [the DataCamp course on Data Manipulation with Pandas](https://app.datacamp.com/learn/courses/data-manipulation-with-pandas), and the optional DataCamp courses on Data Visualization with ggplot2/Matplotlib

In [ ]:
# your code here

(Your interpretation here)

## 1.3 Investigating mechanisms: incarceration rates by charge

Your colleague sees the previous graph and is worried that the gap could be different---either wider or smaller---if you adjust for the fact that prosecutors have discretion in what crimes to charge defendants with. If white defendants are charged with crimes that tend to receive probation rather than incarceration, that could explain some of the gaps.

In the next questions, you'll begin to investigate this.

In [ ]:
# your code here

### 1.3.1 Find the most common offenses (3 points)

First, create a set of 'frequent offenses' that represent (over the entire period) the union of the 10 offenses Black defendant are most likely to be charged with and the 10 offenses white defendants are most likely to be charged with (might be far less than 20 total if there's a lot of overlap in common charges)

Use the `simplified_offense_derived` for this.

**Hint:** To get the unique elements of a list (i.e., remove overlaps), create a `set()`, which only stores unique values (syntax slightly different than with lists).

**Concepts tested and resources:**

- Row subsetting and sorting, as covered in [these lecture slides on data wrangling](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/02_qss20_f22_pandas.pdf), [the class activity on data wrangling](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/00_pandas_datacleaning_blank.ipynb) and its [solutions](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/00_pandas_datacleaning_solutions.ipynb), and [the DataCamp on Data Manipulation with Pandas](https://app.datacamp.com/learn/courses/data-manipulation-with-pandas)

In [ ]:
# your code here

### 1.3.2 Look at incarceration rates (again just whether incarcerated) by race and offense type for these top offenses (3 points)

Print a wide-format version of the resulting table (so each row is an offense type, one col is black incarceration rate for that offense type; another is the white incarceration rate) and interpret. What offenses show the largest disparities in judges being less likely to sentence White defendants to incarceration/more likely to offer those defendants probation?

**Hint:** To create a wide-format version of a table, one option is [`pd.pivot_table()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html). 

**Concepts tested and resources:**

- Row subsetting with logical conditions (e.g., with np.where), as covered in [these lecture slides on data wrangling](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/02_qss20_f22_pandas.pdf) and [the class activity on data wrangling](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/00_pandas_datacleaning_blank.ipynb) and its [solutions](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/00_pandas_datacleaning_solutions.ipynb)

In [ ]:
# your code here

(Your text response here)

### 1.3.3 Examine whether this changes pre and post change to charging threshold for retail theft (10 points)

One important question is not only whether there are disparities by offense type, but also whether these disparities have changed over time.

For instance, the SAO (State Attorney Office) announced in December of 2016 that they would no longer default to charging retail thefts of under \$1,000 as felonies. This change might have (1) decreased disparities or (2) increased disparities, depending on the correlation between race/ethnicity and magnitude of goods stolen (see [this article](https://www.dnainfo.com/chicago/20161215/little-village/kim-foxx-raises-bar-for-retail-theft-felonies/) for more background). 

Focusing on `simplified_offense_derived` == "Retail theft", write a user-defined function that allows you to efficiently: 

- Compare Black-White disparities before and after the change using a two-month bandwidth (so pre is October and November 2016; post is January and February 2017)

- Compare Black-White disparities before and after the change using a four-month bandwidth (so pre is August- November 2016; post is January - April 2017)

- Compare Black-White disparities using an eight-month bandwidth

- Compare Black-White disparities using a twelve-month bandwidth

In other words, the function should compare percentages of defendants incarcerated for retail theft by race. The numerator in the proportions is the # of defendants incarcerated for retail theft, the denom is # of total defendants for retail theft (calculate this separately for each race and separately for before versus after); disparity is the difference in proportions.

Exclude Dec. 2016 as a transition month.

------------------ 

Print a table with the results (any organization is fine as long as it's clear).
 
------------------ 
 
**Concepts tested and resources**:
    
- User-defined functions and list comprehensions, as covered in [these slides](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/03_qss20_f22_userdefinedfunctions.pdf) and [the functions activity](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/01_functions_blank.ipynb) and [its solutions](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/01_functions_solutions.ipynb)

- Row subsetting with logical conditions (e.g., with np.where), as covered in [these lecture slides on data wrangling](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/02_qss20_f22_pandas.pdf) and [the class activity on data wrangling](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/00_pandas_datacleaning_blank.ipynb) and its [solutions](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/00_pandas_datacleaning_solutions.ipynb)

- Visualization, as covered in [this plotnine example code](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/02_plottingexamples_plotnine.ipynb), chapter 4 of [the DataCamp course on Data Manipulation with Pandas](https://app.datacamp.com/learn/courses/data-manipulation-with-pandas), and the optional DataCamp courses on Data Visualization with ggplot2/Matplotlib

**Hints on function:** Your function should take these steps:

1. Create a December 2016 object. 
2. Filter to rows within December 2016 +- that # of months. For example, for the 2-month bandwidth, the "before" period is Oct and Nov 2016; after is Jan and Feb 2017. 
3. Within that filtered dataset, examine Black-White disparities in incarceration before versus after. One shortcut for doing this is to keep the full dataframe together and construct an `is_after` indicator (e.g., using `np.where()`) that takes the value of `True` if after Dec 2016 (and otherwise is False), and then group by that and a categorical race variable. This step produces a single dataframe for each time window--e.g., a dataframe for the 2-month bandwidth, a dataframe for the 4-month one, etc. 
4. Use `pd.concat` to combine those dataframes into a single dataframe.

**Hint on output:** The table you make should have two Black-white disparities per bandwidth: one disparity (e.g., a 10 percentage point difference in incarceration rates) before the policy change and another disparity after (e.g., a 5 percentage point difference in incarceration rates). 

In [ ]:
# your code here

### 1.3.4 Visualize the above (4 points)

Use the table you just made to create a bar chart, where the x axis represents different bandwidths (2, 4, etc); the y axis the size of the Black-White gap, and for each of the x axis points, you have one shaded bar representing "before" the change, another representing "after" the change (make sure that before is ordered before after and the bandwidths are from smallest to largest)

*Note*: for each of the bandwidths include dates spanning the entire month (e.g., for the first, include not only 02-01-2017 but everything up through 02-28-2017; easiest way is for the subsetting to use the rounded `sentenceym_derived`. Also make sure to only include white or black defendants.


**Hints**: 

- Depending on how you calculate/reshape things, you may find [this issue useful for how to collapse column names with a multilevel index](https://stackoverflow.com/questions/24290297/pandas-dataframe-with-multiindex-column-merge-levels) (also may not need it depending on how you structure the code).

- The x-axis on the plot should be a categorical variable, with each of the bandwidths and with separate bars for before vs. after. If you want to change the order of the categories, [check out the `reorder_categories` function in this SO issue](https://stackoverflow.com/questions/38023881/pandas-change-the-order-of-levels-of-factor-type-object).

**Extra credit** (1 point): because the bandwidths have different sample sizes, a better viz incorporates measures of uncertainty. Add standard errors to the points using the formula: $(\dfrac{p(1-p)}{n})^{0.5}$ where N is the number of cases in each bandwidth period.

In [ ]:
# your code here

### 1.3.5 Interpret the results (1 point)

Write a two-sentence interpretation of the results. What might this show about how people on both sides of the issue---those arguing the policy change will narrow disparities; those arguing the change may widen disparities--could support their claims? 

(your interpretation here)

# 2. Extra credit (2 points)

The above question asked about black-white disparities before and after the policy change. Write a new user-defined function that:

- Has an argument(s) to indicate the levels of the `RACE` variable in the original data that will constitute two groups to compare: group 1 and group 2 (eg in one execution of the function, group 1 might be non-Hispanic white; group 2 might be Hispanic and Black; in another execution, group 1 might be Asian; group 2 but might be Hispanic. Note all levels need to be included in a group)
- Can be used to calculate the same bandwidth-specific disparities as above for defendants in those two groups
- Returns a table or plot with the results

**Concepts tested and resources**:
    
- User-defined functions, as covered in [the lecture on functions](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/03_qss20_f22_userdefinedfunctions.pdf) and [the functions activity](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/01_functions_blank.ipynb) and [its solutions](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/01_functions_solutions.ipynb)

In [ ]:
# your code here